In [8]:
pip install delta-spark

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyspark-3.2.4-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip show pyspark

Name: pyspark
Version: 3.1.2
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /opt/spark-3.1.2-bin-hadoop3.2/python
Requires: py4j
Required-by: delta-spark
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pyspark --packages io.delta:delta-core_2.12:1.0.0 \
  --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" \
  --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog"

/opt/spark2/bin/pyspark: line 45: python: command not found
Python 3.6.9 (default, Dec  8 2021, 21:08:43) 
[GCC 8.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
:: loading settings :: url = jar:file:/opt/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/itversity/.ivy2/cache
The jars for the packages stored in: /home/itversity/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0146762b-b61d-4418-8009-7c35c252eb15;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in cen

In [5]:
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
import os
from pyspark.sql import SparkSession

# Path to the downloaded Delta Lake JAR file
delta_jar = "/spark3-jars/delta-core_2.12-1.0.0.jar"

# Initialize the Spark session with Delta Lake support
spark = SparkSession.builder \
    .appName("DeltaApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars", delta_jar) \
    .getOrCreate()

# Verify the Spark session is created
spark.version


In [6]:
# Initialize the Spark session
#delta_jar = "/spark3-jars/delta-core_2.12-1.0.0.jar"

spark = SparkSession.builder \
    .appName("DeltaExample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .getOrCreate()


In [3]:
# Create a sample DataFrame
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
df = spark.createDataFrame(data, ["name", "age"])


In [8]:
column = spark.range(0,7)

In [10]:
column.write.format("delta").save("/delta-test")

In [19]:
column_read = spark.read.format("delta").load("/delta-test")
column_read.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  4|
|  6|
|  7|
|  8|
|  5|
|  3|
|  9|
+---+



In [12]:
column1 = spark.range(5,10)

In [15]:
column1

id
5
6
7
8
9


In [13]:
deltaTable = DeltaTable.forPath(spark, '/delta-test')

In [17]:
deltaTable.alias("target").merge(
    column1.alias("source"),
    "target.id = source.id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [29]:
deltaTable.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  4|
|  6|
|  7|
|101|
|  8|
|102|
|  5|
|103|
|  3|
|100|
|104|
|  9|
+---+



In [21]:
# Create a sample DataFrame
column2 = [("Alice", 30), ("Bob", 45), ("Cathy", 29), ("Ahmed", 26)]
column2 = spark.createDataFrame(data, ["name", "age"])

In [27]:
column5 = spark.range(100,105)

In [28]:
deltaTable.alias("target").merge(
    column5.alias("source"),
    "target.id = source.id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [30]:
# Create a sample DataFrame
data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
df = spark.createDataFrame(data, ["name", "id"])


In [31]:
# Save DataFrame as a Delta table
df.write.format("delta").save("/delta-final11")

In [33]:
df_read = spark.read.format("delta").load("/delta-final11")
df_read.show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
+-----+---+



In [19]:
deltaTable = DeltaTable.forPath(spark, '/delta-final11')

In [35]:
deltaTable.toDF().show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
+-----+---+



In [36]:
# Create a sample DataFrame
data = [("Ahmed", 1), ("Bob", 2), ("Cathy", 3), ("mooo", 4)]
df_new = spark.createDataFrame(data, ["name", "id"])


In [37]:
deltaTable.alias("target").merge(
    df_new.alias("source"),
    "target.id = source.id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [20]:
deltaTable.toDF().show()

+-----+---+
| name| id|
+-----+---+
|Cathy|  3|
|Ahmed|  1|
| mooo|  4|
|  Bob|  2|
+-----+---+



In [32]:
deltaTable.history()

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
2,2024-07-19 16:06:...,null,null,WRITE,{mode -> Overwrit...,null,null,null,1,null,false,"{numFiles -> 1, n...",null
1,2024-07-19 13:07:...,null,null,MERGE,{predicate -> (ta...,null,null,null,0,null,false,{numTargetRowsCop...,null
0,2024-07-19 13:04:...,null,null,WRITE,{mode -> ErrorIfE...,null,null,null,null,null,true,"{numFiles -> 2, n...",null


In [61]:
df_read = spark.read.format("delta").option("versionAsof",0).load("/delta-final11")
df_read.show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
+-----+---+



In [62]:
df_read = spark.read.format("delta").option("versionAsof",1).load("/delta-final11")
df_read.show()

+-----+---+
| name| id|
+-----+---+
|Cathy|  3|
|Ahmed|  1|
| mooo|  4|
|  Bob|  2|
+-----+---+



In [10]:
fi =spark.read.parquet("/delta-final11/part-00069-98dd1980-1c7f-4dec-9b36-fd00a1e0bb8b-c000.snappy.parquet").show()

+-----+---+
| name| id|
+-----+---+
|Ahmed|  1|
+-----+---+



In [9]:
fi =spark.read.parquet("/delta-final11/part-00128-d6362f93-c607-467e-9e1b-766835d9d29f-c000.snappy.parquet").show()

+----+---+
|name| id|
+----+---+
| Bob|  2|
+----+---+



In [11]:
fi =spark.read.parquet("/delta-final11/part-00107-3beafe8f-775e-41a2-85fc-8282fb10caa9-c000.snappy.parquet").show()

+-----+---+
| name| id|
+-----+---+
|Cathy|  3|
+-----+---+



In [12]:
fi =spark.read.parquet("/delta-final11/part-00000-6ecfb59d-1616-4baf-8d7d-cd0753677be9-c000.snappy.parquet").show()

+----+---+
|name| id|
+----+---+
+----+---+



In [14]:
fi =spark.read.parquet("/delta-final11/part-00140-4037259f-1916-4e13-acb7-e720dd63d9f1-c000.snappy.parquet").show()

+----+---+
|name| id|
+----+---+
|mooo|  4|
+----+---+



In [15]:
fi =spark.read.parquet("/delta-final11/part-00000-3131d693-7984-4b4d-ab27-a2634effe9b1-c000.snappy.parquet").show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
+-----+---+



In [16]:
fi =spark.read.parquet("/delta-final11/part-00001-b279eff0-2043-4640-8617-deb2ca671fec-c000.snappy.parquet").show()

+-----+---+
| name| id|
+-----+---+
|  Bob|  2|
|Cathy|  3|
+-----+---+



In [23]:
deltaTable.optimize().executeCompaction()

AttributeError: 'DeltaTable' object has no attribute 'optimize'

In [22]:
from delta.tables import *

In [29]:
path = "/delta-final11"
numFiles = 1

(
    spark.read.format("delta")
    .load(path)
    .repartition(numFiles)
    .write.option("dataChange", "false")
    .format("delta")
    .mode("overwrite")
    .save(path)
)

In [31]:
fi =spark.read.parquet("/delta-final11/part-00000-881fe57e-ed6b-48ed-8405-630987b3c5f8-c000.snappy.parquet").show()

+-----+---+
| name| id|
+-----+---+
|  Bob|  2|
|Cathy|  3|
|Ahmed|  1|
| mooo|  4|
+-----+---+



In [ ]:
data_new = [("Alice", 30), ("Bob", 45), ("Cathy", 29), ("Ahmed", 26)]
df_new = spark.createDataFrame(data, ["name", "age"])

In [4]:
# Save DataFrame as a Delta table
df.write.format("delta").save("/delta-final22")


In [7]:
df

name,age
Alice,34
Bob,45
Cathy,29


In [6]:
df_read = spark.read.format("delta").load("/delta-final22")
df_read.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|Cathy| 29|
+-----+---+



In [35]:
deltaTable = DeltaTable.forPath(spark, '/delta-final2')

In [ ]:
deltaTable.alias("target").merge(
    df_new.alias("source"),
    "target.name = source.name").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [54]:
# Perform the merge operation
deltaTable.alias("target").merge(
    df_new.alias("source"),
    "target.name = source.name"
).whenMatchedUpdate(set={
    "age": col("source.age")
}).whenNotMatchedInsert(values={
    "name": col("source.name"),
    "age": col("source.age")
}).execute()

In [58]:
deltaTable.toDF().show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|Cathy| 29|
|  Bob| 45|
+-----+---+



In [ ]:
merge = (
    deltaTable.alias("Dest")
    .merge(
        source_df.alias("Source"),
        """
        (Source.name == Dest.name)

        """,
    )
    .whenMatchedUpdate(condition="Dest.Client IS NULL", set={"Client": col("Source.Client")})
    .whenMatchedUpdate(condition="Dest.Description IS NULL", set={"Description": col("Source.Description")})
    ...
    ...
    whenNotMatchedInsertAll()
)

merge.execute()

In [62]:
result_df = spark.read.format("delta").load("/delta-final2")
result_df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|Cathy| 29|
|  Bob| 45|
+-----+---+



In [61]:
deltaTable.toDF().show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|Cathy| 29|
|  Bob| 45|
+-----+---+



In [ ]:
spark.stop()

In [65]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     |################################| 9.5 MB 3.3 MB/s            
     |################################| 14.8 MB 287 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
